# Description
Welcome to Prostate cANcer graDe Assessment (PANDA) Challenge. The task of this competition is classification of images with cancer tissue. The main challenge of this task is dealing with images of extremely high resolution and large areas of empty space. So, effective locating the areas of concern and zooming them in would be the key to reach high LB score.

In this competition I found a number of public kernels performing straightforward rescaling the input images to square. However, for this particular data such an approach is not very efficient because the aspect ratio and size of provided images are not consistent and vary in a wide range. As a result, the input images are deformed to large extend in a not consistent manner uppon rescaling that limits the ability of the model to learn. Moreover, the input consists of large empty areas leading to inefficient use of GPU memory and GPU time.

In this kernel I propose an alternative approach based on **Concatenate Tile pooling**. Instead of passing an entire image as an input, N tiles are selected from each image based on the number of tissue pixels (see [this kernel](https://www.kaggle.com/iafoss/panda-16x128x128-tiles) for description of data preparation and the [corresponding dataset](https://www.kaggle.com/iafoss/panda-16x128x128-tiles-data)) and passed independently through the convolutional part. The outputs of the convolutional part is concatenated in a large single map for each image preceding pooling and FC head (see image below). Since any spatial information is eliminated by the pooling layer, the Concat Tile pooling approach is nearly identical to passing an entire image through the convolutional part, excluding predictions for nearly empty regions, which do not contribute to the final prediction, and shuffle the remaining outputs into a square map of smaller size. Below I provide just a basic kernel only illustrating this approach. In my first trial I got 0.76 LB score, top 2 at the moment, and I believe it could be easily boosted to 0.80+. I hope you would enjoy my kernel, and please also check my submission kernel implementing the tile based approach.

![](https://i.ibb.co/hF6LRVm/TILE.png)

In [ ]:
!pip install -q efficientnet_pytorch   

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from sklearn.metrics  import confusion_matrix
import fastai
from fastai.vision import *
from fastai.callbacks import SaveModelCallback,ReduceLROnPlateauCallback
import os
import numpy as np
import matplotlib.pyplot as py
from tqdm import tqdm
from sklearn.metrics import cohen_kappa_score
from radam import *
from csvlogger import *
from mish_activation import *
import warnings
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings("ignore")
from efficientnet_pytorch import EfficientNet
fastai.__version__

In [ ]:
import os
os.listdir('../input/panda-dataset-medium-32-256-256/train_medium_32_256_256');

In [ ]:
sz = 256
bs = 2
nfolds = 4
SEED = 2020
N = 32 #number of tiles per image
TRAIN = '../input/panda-dataset-medium-32-256-256/train_medium_32_256_256/'
LABELS = '../input/prostate-cancer-grade-assessment/train.csv'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

# Data

Use stratified KFold split.

In [ ]:
df = pd.read_csv(LABELS).set_index('image_id')
files = set([p[:32] for p in os.listdir(TRAIN)])
df = df.loc[files]
df = df.reset_index()
splits = StratifiedKFold(n_splits=nfolds, random_state=SEED, shuffle=True)
splits = list(splits.split(df,df.isup_grade))
folds_splits = np.zeros(len(df)).astype(np.int)
for i in range(nfolds): folds_splits[splits[i][1]] = i
df['split'] = folds_splits
df.head()

In [ ]:
df['split'].hist()

# Binning Labels


In [ ]:
def bin(x):
    n=np.zeros(6).astype(np.float32)
    n[:x]=1
    return n

df['ord_label']=df['isup_grade'].apply(bin)

p1=pd.DataFrame(df['ord_label'].values)

l=[]
for i in range(p1.shape[0]):
    l.extend(p1.iloc[i,:].values[0])

n=np.array(l).reshape(p1.shape[0],6)

p2=pd.DataFrame(n)

df=pd.concat([df,p2],axis=1)

df.columns=['image_id', 'data_provider',    'isup_grade', 'gleason_score',
               'split',     'ord_label','ord_0','ord_1','ord_2','ord_3','ord_4','ord_5']

In [ ]:
df.head()

Check [this kernel](https://www.kaggle.com/iafoss/panda-16x128x128-tiles) for image stats. Since I use zero padding and background corresponds to 255, I invert images as 255-img when load them. Therefore, the mean value is computed as '1 - val'.

In [ ]:
mean = torch.tensor([1.0-0.90949707, 1.0-0.8188697, 1.0-0.87795304])
std = torch.tensor([0.36357649, 0.49984502, 0.40477625])

The code below creates ImageItemList capable of loading multiple tiles of an image. It is specific for fast.ai, and pure Pytorch code would be much simpler.

In [ ]:
def open_image(fn:PathOrStr, div:bool=True, convert_mode:str='RGB', cls:type=Image,
        after_open:Callable=None)->Image:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning) # EXIF warning from TiffPlugin
        x = PIL.Image.open(fn).convert(convert_mode)
    if after_open: x = after_open(x)
    x = pil2tensor(x,np.float32)
    if div: x.div_(255)
    return cls(1.0-x) #invert image for zero padding

class MImage(ItemBase):
    def __init__(self, imgs):
        self.obj, self.data = \
          (imgs), [(imgs[i].data - mean[...,None,None])/std[...,None,None] for i in range(len(imgs))]
    
    def apply_tfms(self, tfms,*args, **kwargs):
        for i in range(len(self.obj)):
            self.obj[i] = self.obj[i].apply_tfms(tfms, *args, **kwargs)
            self.data[i] = (self.obj[i].data - mean[...,None,None])/std[...,None,None]
        return self
    
    def __repr__(self): return f'{self.__class__.__name__} {img.shape for img in self.obj}'
    def to_one(self):
        img = torch.stack(self.data,1)
        img = img.view(3,-1,N,sz,sz).permute(0,1,3,2,4).contiguous().view(3,-1,sz*N)
        return Image(1.0 - (mean[...,None,None]+img*std[...,None,None]))

class MImageItemList(ImageList):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def __len__(self)->int: return len(self.items) or 1 
    
    def get(self, i):
        fn = Path(self.items[i])
        fnames = [Path(str(fn)+'_'+str(i)+'.png')for i in range(N)]
        imgs = [open_image(fname, convert_mode=self.convert_mode, after_open=self.after_open)
               for fname in fnames]
        return MImage(imgs)

    def reconstruct(self, t):
        return MImage([mean[...,None,None]+_t*std[...,None,None] for _t in t])
    
    def show_xys(self, xs, ys, figsize:Tuple[int,int]=(300,50), **kwargs):
        rows = min(len(xs),8)
        fig, axs = plt.subplots(rows,1,figsize=figsize)
        for i, ax in enumerate(axs.flatten() if rows > 1 else [axs]):
            xs[i].to_one().show(ax=ax, y=ys[i], **kwargs)
        plt.tight_layout()

In [ ]:
def get_data(fold=0):
    return (MImageItemList.from_df(df, path='.', folder=TRAIN, cols='image_id')
      .split_by_idx(df.index[df.split == fold].tolist())
      .label_from_df(cols=['isup_grade'],)
      .transform(get_transforms(flip_vert=False,),size=sz,padding_mode='zeros')
      .databunch(bs=bs,num_workers=4))

data = get_data(3)
data.show_batch()

# Unet 
https://github.com/milesial/Pytorch-UNet/blob/master/unet/

In [ ]:
import torch.nn as nn
import math

class Selayer(nn.Module):

    def __init__(self, inplanes):
        super(Selayer, self).__init__()
        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        self.conv1 = nn.Conv2d(inplanes, int(inplanes / 16), kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(int(inplanes / 16), inplanes, kernel_size=1, stride=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):

        out = self.global_avgpool(x)

        out = self.conv1(out)
        out = self.relu(out)

        out = self.conv2(out)
        return x * out


class BottleneckX(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, cardinality, stride=1, downsample=None):
        super(BottleneckX, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes * 2, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes * 2)

        self.conv2 = nn.Conv2d(planes * 2, planes * 2, kernel_size=3, stride=stride,
                               padding=1, groups=cardinality, bias=False)
        self.bn2 = nn.BatchNorm2d(planes * 2)

        self.conv3 = nn.Conv2d(planes * 2, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)

        self.selayer = Selayer(planes * 4)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out = self.selayer(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class SEResNeXt(nn.Module):

    def __init__(self, block, layers, num_classes,cardinality=32):
        super(SEResNeXt, self).__init__()
        self.cardinality = cardinality
        self.inplanes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        nc = self.layer4[-1].selayer.conv2.weight.shape[0]
        self.head = Head(n_classes=num_classes,in_features=nc)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                
        

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, self.cardinality, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, self.cardinality))

        return nn.Sequential(*layers)

    def forward(self, *x):
        shape=x[0].shape
        n_tiles=12#len(x)
        x=torch.stack(x,1).view(-1,shape[1],shape[2],shape[3])
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        print(f'After pooling: {x.size()}')
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        print(f'After layer4: {x.size()}')
        print(x)
        shape1=x.size()
        x=x.view(-1,shape1[1],n_tiles*shape1[2],shape1[2])
        x1 = self.head(x)
        return x1
class Head(nn.Module):
    def __init__(self,n_classes,in_features):
        super(Head,self).__init__()
        self.head=nn.Sequential(AdaptiveConcatPool2d(),Flatten(),nn.Linear(2*in_features,n_classes))
    def forward(self,x):
        return self.head(x)

def se_resnext50(**kwargs):
    """Constructs a SE-ResNeXt-50 model.
    Args:
    num_classes = 1000 (default)
    """
    model = SEResNeXt(block=BottleneckX, layers=[3, 4, 6, 3], num_classes=6)
    return model

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1,stride=(2,2)),
            nn.BatchNorm2d(mid_channels),
            Mish(),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1,stride=(2,2)),
            nn.BatchNorm2d(out_channels),
            Mish()
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.m=EfficientNet.from_pretrained(num_classes=n_classes,model_name='efficientnet-b0')
        nc=list(self.m.children())[-2].in_features
        self.head1=Head(in_features=nc,n_classes=n_classes)
    def print(self,x):
        print(x[0].size())
        fig,ax=py.subplots(4,3)
        j=0
        for ax1 in ax:
            for ax2 in ax1:
                print(x[j,0,:,:].detach().numpy().size())
                ax2.imshow(x[j,0,:,:].detach().numpy())
                j+=1
    
    def forward(self, *x):
        shape=x[0].shape
        n_tiles=len(x)
        x=torch.stack(x,1).view(-1,shape[1],shape[2],shape[3])
        x1 = self.m.extract_features(x)
        shape1=x1.size()
        x1=x1.view(-1,shape1[1],n_tiles*shape1[2],shape1[2])#shape1[2]
        img = self.head1(x1)
        shape1=img.size()
        return img
class Head(nn.Module):
    def __init__(self,n_classes,in_features):
        super(Head,self).__init__()
        self.head=nn.Sequential(AdaptiveConcatPool2d(),Flatten(),nn.Linear(2*in_features,n_classes))
    def forward(self,x):
        return self.head(x)

In [ ]:
class SaveModelCallback(TrackerCallback):
    "A `TrackerCallback` that saves the model when monitored quantity is best."
    def __init__(self, learn:Learner, monitor:str='valid_loss', mode:str='auto',
                 every:str='improvement', name:str='bestmodel'):
        super().__init__(learn, monitor=monitor, mode=mode)
        self.every,self.name = every,name
        if self.every not in ['improvement', 'epoch']:
            warn(f'SaveModel every {self.every} is invalid, falling back to "improvement".')
            self.every = 'improvement'
                 
    def jump_to_epoch(self, epoch:int)->None:
        try: 
            self.learn.load(f'{self.name}_{epoch-1}', purge=False)
            print(f"Loaded {self.name}_{epoch-1}")
        except: print(f'Model {self.name}_{epoch-1} not found.')

    def on_epoch_end(self, epoch:int, **kwargs:Any)->None:
        "Compare the value monitored to its best score and maybe save the model."
        if self.every=="epoch": 
            torch.save(learn.model.state_dict(),f'{self.name}_{epoch}.pth')
        else: #every="improvement"
            current = self.get_monitor_value()
            if current is not None and self.operator(current, self.best):
                self.best = current
                torch.save(learn.model.state_dict(),f'{self.name}.pth')

    def on_train_end(self, **kwargs):
        "Load the best model."
        if self.every=="improvement" and os.path.isfile(f'{self.name}.pth'):
            self.model.load_state_dict(torch.load(f'{self.name}.pth'))

In [ ]:
class CutMixUpCallback(LearnerCallback):
    "Callback that creates the mixed-up input and target."
    def __init__(self, learn, alpha=0.4, cutmix_alpha=1.0,cutmix_prob=0.5):
        super().__init__(learn)
        self.alpha = alpha
        self.cutmix_prob = cutmix_prob
        self.cutmix_alpha = cutmix_alpha

    def on_train_begin(self, **kwargs):
        self.learn.loss_func = MixLoss(self.learn.loss_func)
        #pass
    def on_batch_begin(self, last_input, last_target, train, **kwargs):
        last_input=torch.stack(last_input,1)
        last_target=last_target.view(-1,1)
        if not train: 
            return

        # Implement cutmix
        if np.random.rand() < self.cutmix_prob:
            B, W, H = last_input.size(0), last_input.size(1), last_input.size(2)
            indices = torch.randperm(B)
            Lam = []
            last_input_ = last_input.clone()
            for i in range(B):
                lam = np.random.beta(self.cutmix_alpha, self.cutmix_alpha)
                cut_rat = np.sqrt(1. - lam)
                cut_w = np.int(W * cut_rat)
                cut_h = np.int(H * cut_rat)

                # uniform
                cx = np.random.randint(W)
                cy = np.random.randint(H)

                bbx1 = np.clip(cx - cut_w // 2, 0, W)
                bby1 = np.clip(cy - cut_h // 2, 0, H)
                bbx2 = np.clip(cx + cut_w // 2, 0, W)
                bby2 = np.clip(cy + cut_h // 2, 0, H)
                last_input[i, bbx1:bbx2, bby1:bby2] = last_input_[indices[i], bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                Lam.append(1 - (bbx2 - bbx1) * (bby2 - bby1) / (W * H))
            Lam = torch.tensor(Lam).cuda()
            new_input = last_input
            new_target = torch.cat([last_target.float(), last_target[indices].float(), Lam[:,None].float()], 1).long()
            
        else:
            lambd = np.random.beta(self.alpha, self.alpha, last_target.size(0))
            lambd = np.concatenate([lambd[:,None], 1-lambd[:,None]], 1).max(1)
            lambd = last_input.new(lambd)
            shuffle = torch.randperm(last_target.size(0)).to(last_input.device)
            # No stack x, compute x
            out_shape = [lambd.size(0)] + [1 for _ in range(len(last_input.shape) - 1)]
            new_input = (last_input * lambd.view(out_shape) + last_input[shuffle] * (1-lambd).view(out_shape))
            # Stack y
            new_target = torch.cat([last_target.float(), last_target[shuffle].float(), lambd[:,None].float()], 1).long()
        new_input=new_input.permute(1,0,2,3,4)
        return {'last_input': list(new_input), 'last_target': new_target}  

    def on_train_end(self, **kwargs):
        self.learn.loss_func = self.learn.loss_func.get_old()
        #pass
class MixLoss(Module):
    "Adapt the loss function `crit` to go with mixup &amp; cutmix."
    def __init__(self, crit, reduction='mean'):
        super().__init__()
        if hasattr(crit, 'reduction'): 
            self.crit = crit
            self.old_red = crit.reduction
            setattr(self.crit, 'reduction', 'none')
        else: 
            self.crit = partial(crit, reduction='none')
            self.old_crit = crit
        self.reduction = reduction

    def forward(self, output, target):
        if len(target.shape) == 2 and target.size(-1)==3:
            loss1, loss2 = self.crit(output,target[:,0]), self.crit(output,target[:,1])
            d = loss1 * target[:,-1] + loss2 * (1-target[:,-1])
        else:  
            target=target.long()
            d = self.crit(output, target)

        if self.reduction == 'mean':    
            return d.mean()
        elif self.reduction == 'sum':   
            return d.sum()
        return d

    def get_old(self):
        if hasattr(self, 'old_crit'):  return self.old_crit
        elif hasattr(self, 'old_red'): 
            setattr(self.crit, 'reduction', self.old_red)
            return self.crit


# Loss Function

In [ ]:
class ModifiedCrossEntropy(nn.Module):
    def __init__(self):
        super(ModifiedCrossEntropy,self).__init__()
    def forward(self,input,target,reduction='mean'):
        target=target.long()
        input=input.float()
        return loss/len(input[0])
    

# Metrics

In [ ]:
def Kappa(output,target):
    output=output.cpu()
    target=target.cpu()
    output=output.sigmoid().sum(1).round()
    target=target.sum(1)
    print(output)
    print(target)
    print(cohen_kappa_score(output,target,weights='quadratic'))
    return torch.tensor(cohen_kappa_score(output,target,weights='quadratic'))

# Training

In [ ]:

model=UNet(3,6)
learn = Learner(data, model, loss_func=nn.CrossEntropyLoss(), opt_func=Adam, 
                metrics=[KappaScore(weights='quadratic')],model_dir='.')

logger = CSVLogger(learn, f'log')
learn.clip_grad = 1.0
learn.split([model.head1])

learn.unfreeze()
learn.lr_find()

learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(7,max_lr=slice(4e-4,9e-4),
                    callbacks=
                    [logger,SaveModelCallback(learn,name='bestmodel',every='epoch',monitor='KappaScore'),
                     ReduceLROnPlateauCallback(learn,),])

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.export()